Implementing the full Skip-gram and CBOW models from scratch in PyTorch involves defining the architecture, the custom loss function (usually **Negative Sampling** for efficiency), and the training loop.

Since the basic architecture for both is a very shallow neural network, the implementations are quite similar. We'll use **Negative Sampling** as the loss function, as standard Softmax is computationally prohibitive for large vocabularies.

-----

## 🛠️ 1. Setup and Helper: Negative Sampling Loss

Both Skip-gram and CBOW rely on the same fundamental mechanism for efficient training: **Negative Sampling Loss**. This avoids calculating gradients for the entire vocabulary.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NegativeSamplingLoss(nn.Module):
    """
    Implements the Negative Sampling loss function for Word2Vec.
    It maximizes the similarity of the target/context pair (positive sample)
    and minimizes the similarity of the target/negative pairs.
    """
    def __init__(self):
        super().__init__()

    def forward(self, positive_score, negative_score):
        # 1. Positive Loss: Maximize the probability of the correct pair (Sigmoid close to 1)
        # We maximize log(sigma(x)), which is equivalent to minimizing -log(sigma(x))
        positive_loss = F.logsigmoid(positive_score)

        # 2. Negative Loss: Minimize the probability of incorrect pairs (Sigmoid close to 0)
        # We maximize log(sigma(-x)), which is equivalent to minimizing -log(sigma(-x))
        negative_loss = F.logsigmoid(-negative_score).sum(1)

        # Total loss is the mean of the combined positive and negative losses
        # Note: We take the negative because we minimize the negative log likelihood.
        return -(positive_loss + negative_loss).mean()

# Loss module initialized globally
criterion = NegativeSamplingLoss()

-----

## 2\. Skip-gram Implementation (Target -\> Context)

The Skip-gram model predicts the context words from a single target word.

In [ ]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Input Embedding Matrix (the word vectors we learn)
        self.input_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Output Embedding Matrix (used to score against context/negative words)
        self.output_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Initialization helps stability
        self.input_embeddings.weight.data.uniform_(-1, 1)
        self.output_embeddings.weight.data.uniform_(-1, 1)

    def forward(self, target_word_indices, context_word_indices, negative_word_indices):
        """
        Args:
            target_word_indices: Indices of the central words (B,)
            context_word_indices: Indices of the positive context words (B,)
            negative_word_indices: Indices of the negative samples (B, N)
        """
        # 1. Get Embeddings for Target and Context (Input Embeddings)
        target_v = self.input_embeddings(target_word_indices) # (B, E)
        context_v = self.output_embeddings(context_word_indices) # (B, E)

        # 2. Get Embeddings for Negative Samples (Output Embeddings)
        negative_v = self.output_embeddings(negative_word_indices) # (B, N, E)

        # 3. Calculate Positive Score (Target . Context)
        # (B, E) @ (B, E) -> (B, 1) -> Sigmoid is applied in the loss function
        positive_score = torch.sum(target_v * context_v, dim=1)

        # 4. Calculate Negative Scores (Target . Negative)
        # Need to reshape target_v for broadcasting: (B, 1, E)
        target_v = target_v.unsqueeze(1)

        # (B, 1, E) @ (B, N, E) -> (B, N)
        negative_score = torch.bmm(target_v, negative_v.transpose(1, 2)).squeeze(1)

        return positive_score, negative_score

-----

## 3\. CBOW Implementation (Context -\> Target)

The CBOW model predicts the target word from an **average** of its context words.

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Input Embedding Matrix
        self.input_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Output Embedding Matrix
        self.output_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.input_embeddings.weight.data.uniform_(-1, 1)
        self.output_embeddings.weight.data.uniform_(-1, 1)

    def forward(self, context_word_indices, target_word_indices, negative_word_indices):
        """
        Args:
            context_word_indices: Indices of the context words (B, C) where C is context size * 2
            target_word_indices: Indices of the target words (B,)
            negative_word_indices: Indices of the negative samples (B, N)
        """
        # 1. Get Embeddings for Context Words (Input Embeddings)
        context_v = self.input_embeddings(context_word_indices) # (B, C, E)

        # 2. Average the Context Embeddings to get the Context Vector (X_c)
        # This is the 'Bag-of-Words' step.
        context_vector = torch.mean(context_v, dim=1) # (B, E)

        # 3. Get Embeddings for Target and Negative Samples (Output Embeddings)
        target_v = self.output_embeddings(target_word_indices) # (B, E)
        negative_v = self.output_embeddings(negative_word_indices) # (B, N, E)

        # 4. Calculate Positive Score (Context Vector . Target)
        # (B, E) * (B, E) -> (B, 1)
        positive_score = torch.sum(context_vector * target_v, dim=1)

        # 5. Calculate Negative Scores (Context Vector . Negative)
        context_vector = context_vector.unsqueeze(1) # (B, 1, E)

        # (B, 1, E) @ (B, N, E) -> (B, N)
        negative_score = torch.bmm(context_vector, negative_v.transpose(1, 2)).squeeze(1)

        return positive_score, negative_score

### Note on Training Data

Both implementations require that your data pipeline generates the following tuples for each batch:

  * **Skip-gram:** `(target_word, context_word, [negative_samples])`
  * **CBOW:** `([context_words], target_word, [negative_samples])`

This data structure is crucial for the models to learn correctly using the negative sampling loss function.